# Corner detection

Pour détecter les coins des objets dans une image, on peut commencer par déterminer les lieux où deux contours se joignent.
Il existe toutefois d'autres méthodes, parmi lesquelles : 
* le détecteur de Moravec [Moravec 1980],
* le détecteur de Harris [Harris 1988].

```{figure} figs/corner-example.png
---
width: 250px
name: F:feature:corners:example
```

## Détecteur de Moravec

Le principe de ce détecteur est d'observer les changements d'un patch (une sous-image)
que l'on décale d'un pixel dans toutes les directions, autour d'un pixel d'intérêt.
Il y a quatre directions : horizontalement, verticalement, et selon les deus diagonales.
Si les changements sont importants, alors le patch est centré sur un coin.

```{figure} figs/feature-moravec.png
---
width: 80%
name: F:feature:corners:moravec
---
Principe du détecteur de Moravec.
De gauche à droite :
sur une zone plate, de petits décalages du patch (en rouge) ne provoquent pas de changement ;
sur un contour, on n'observe des changements que dans une seule direction ;
autour d'un coin, il y a des changements significatifs dans toutes les directions.
```

Mathématiquement, le changement est caractérisé en chaque pixel $(m,n)$ de l'image par
$E_{m,n}(x,y)$ qui représente la différence entre les patchs pour un décalage $(x,y)$ :

$$
  \forall m,n,x,y \qquad E_{m,n}(x,y) = \sum_{u,v} w_{m,n}(u,v) \big[f(u+x,v+y)-f(u,v)\big]^2
$$

où :

* $x$ et $y$ représentent les décalages dans les quatre directions :

  $$
  (x,y) \in \{(1,0),\,(1,1),\,(0,1),\,(-1,1)\},
  $$
  
* $w_{m,n}$ est une fenêtre rectangulaire autour du pixel ($m,n)$,

* $\big[f(u+x,v+y)-f(u,v)\big]^2$ est la différence entre le patch $f(u,v)$ et le patch décalé $f(u+x,v+y)$,

En chaque pixel $(m,n)$, le minimum de $E_{m,n}(x,y)$ dans les quatre directions est conservé et noté $F_{m,n}$.
Finalement, les coins détectés correspondent aux maxima locaux de $F_{m,n}$,
c'est-à-dire aux pixels $(m,n)$ où la plus petite valeur de $E_{m,n}(x,y)$ est grande.

<!-- exemple -->

Il s'avère que le détecteur de Moravec a plusieurs limites.
Premièrement, $w$ est une fenêtre binaire et donc le détecteur considère avec le même poids tous les pixels dans la fenêtre.
Lorsque le bruit dans l'image est important, cela peut amener à de fausses détections de coins.
Deuxièmement, seuls quatre directions sont considérées.
Troisièment, le détecteur reste très sensible aux contours car seul le minimum de $E$ est considéré.
Pour ces raisons, Harris a proposé un détecteur qui répond à ces limites.

## Détecteur de Harris

Pour éviter une réponse bruitée, la fenêtre rectangulaire $w$ du détecteur de Moravec est remplacée par une fenêtre $w$ gaussienne
dans l'expression de $E_{m,n}(x,y)$.

Pour étendre le détecteur de Moravec à toutes les directions, sans se limiter aux quatre directions initiales,
un développement en série de Taylor est effectué sur le patch décalé $f(u+x,v+y)$ :

$$
  f(u+x,v+y) \approx f(u,v) + x \,\partial_x f(u,v) + y \,\partial_y f(u,v).
$$

Par conséquent :

$$
E_{m,n}(x,y) \approx \sum_{u,v} w_{m,n}(u,v) \big[ x \,\partial_x f(u,v) + y \,\partial_y f(u,v) \big]^2
$$

Cette expression peut s'écrire sous la forme matricielle suivante :


$$
E_{m,n}(x,y) \approx
\begin{pmatrix} x & y \end{pmatrix}
M
\begin{pmatrix} x \\ y \end{pmatrix}
$$

où

$$
M = \sum_{u,v} w_{m,n}(u,v)
\begin{pmatrix}
  (\partial_x f)^2  &  \partial_x f \,\partial_y f \\
  \partial_x f \,\partial_y f  &  (\partial_y f)^2 \\
\end{pmatrix}
$$

Enfin, la dernière limite du détecteur de Moravec peut être évitée en considérant une nouvelle mesure de la présence d'un coin :
on peut obtenir d'autres informations sur le changement d'intensité dans la fenêtre
en analysant les valeurs propres $\lambda_1$ et $\lambda_2$ de la matrice $M$ ({numref}`F:feature:corners:eigenvectors`).
En effet, la présence d'un coin est attestée lorsque les dérivées de $f$ sont très grandes,
et alors $M$ possède de grands coefficients, et ses valeurs propres sont également très grandes.

```{figure} figs/feature-30.png
---
width: 250px
name: F:feature:corners:eigenvectors
---
Décision à prendre sur le patch considéré quant à la valeur des valeurs propres.
```

<!-- Introduire comme illustration : figs/toto-16.png et figs/toto-17.png -->

Le calcul des valeurs propres de $M$ pouvant être difficile, une alternative est de calculer :

$$
R = \mathrm{det}(M) - k (\mathrm{trace}(M))^2 = \lambda_1 \lambda_2 - k(\lambda_1 + \lambda_2)^2
$$
avec $0,04 < k < 0,06$.

Ainsi, les valeurs de $R$ sont faibles dans une région plate, négatives sur un contour et positives sur un coin ({numref}`F:feature:corners:R`).

```{figure} figs/feature-31.png
---
width: 250px
name: F:feature:corners:R
---
Décision à prendre sur le patch considéré quant à la valeur de $R$.
```

<!-- Rajouter l'échelle de couleur sur R -->

L'application du détecteur de Harris est illustré sur l'exemple de la {numref}`F:feature:corners:harris`


```{figure} figs/feature-32.png
---
width: 100%
name: F:feature:corners:harris
---
Résultat du détecteur de Harris.
Ces trois images binaires représentent les valeurs négatives (contours), faibles (zones plates) et positives (coins) du coefficient $R$.
```